In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

# langsmith
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

# model
model = init_chat_model(
    "openai:qwen3-max",
    base_url=os.getenv('BASE_URL'),
    api_key=os.getenv('API_KEY')
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,      # ISO format: "2024-01-15T14:00:00"
    end_time: str,        # ISO format: "2024-01-15T15:00:00"
    attendees: list[str], # email addresses
    location: str = ""
) -> str:
    """Create a calendar event. Requires exact ISO datetime format."""
    # Stub: In practice, this would call Google Calendar API, Outlook API, etc.
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"

@tool
def send_email(
    to: list[str], # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    # Stub: In practice, this would call SendGrid, Gmail API, etc.
    return f"Email sent to {', '.join(to)} - Subject: {subject}"

def get_available_time_slots(
    attendees: list[str],
    date: str, # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """Check calendar availability for given attendees on a specific date."""
    # Stub: In practice, this would query calendar APIs
    return ["09:00", "14:00", "16:00"]
    

In [3]:
from langchain.agents import create_agent

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
)

In [4]:
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

for step in calendar_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================

I'll schedule a team meeting for next Tuesday at 2pm for 1 hour. Let me create that calendar event for you.
Tool Calls:
  create_calendar_event (call_cb1a5c0d3c694978a86d8af5)
 Call ID: call_cb1a5c0d3c694978a86d8af5
  Args:
    title: Team Meeting
    start_time: 2024-12-31T14:00:00
    end_time: 2024-12-31T15:00:00
    attendees: ['user']
================================= Tool Message =================================
Name: create_calendar_event

Event created: Team Meeting from 2024-12-31T14:00:00 to 2024-12-31T15:00:00 with 1 attendees
================================== Ai Message ==================================

Perfect! I've successfully scheduled your team meeting for next Tuesday (December 31st, 2024) from 2:00 PM to 3:00 PM. The event has been created with you as the attendee.


In [5]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [6]:
query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_1078e82512e340a6aa6fa011)
 Call ID: call_1078e82512e340a6aa6fa011
  Args:
    to: ['design-team@example.com']
    subject: Reminder: Please Review New Mockups
    body: Hi Team,

This is a friendly reminder to please review the new mockups at your earliest convenience. Your feedback is important to keep the project moving forward.

Thanks!
================================= Tool Message =================================
Name: send_email

Email sent to design-team@example.com - Subject: Reminder: Please Review New Mockups
================================== Ai Message ==================================

I've sent the email to the design team at design-team@example.com with the subject "Reminder: Please Review New Mockups." The message asks them to review the new mockups and provide their feedback.


In [7]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })

    return result["messages"][-1].text

@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """

    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })

    return result["messages"][-1].text

In [8]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
)

supervisor_agent = create_agent(
    model,
    tools = [schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
)

In [9]:
query = "Schedule a team standup for tomorrow at 9am"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_2951548be2454e6fb469a495)
 Call ID: call_2951548be2454e6fb469a495
  Args:
    request: team standup tomorrow at 9am
================================= Tool Message =================================
Name: schedule_event

I'll help you schedule a team standup for tomorrow at 9am. Let me create that calendar event for you.

To do this properly, I need to know who should be included as attendees for the standup meeting. Could you please provide the list of team members who should attend?
================================== Ai Message ==================================

I can schedule the team standup for tomorrow at 9am, but I need to know who should be invited to the meeting. Could you please provide the names or email addresses of the team members who should attend the standup?


In [10]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================

I'll help you schedule the meeting with the design team and send them an email reminder about reviewing the mockups.
Tool Calls:
  schedule_event (call_1250577b098344d598f02610)
 Call ID: call_1250577b098344d598f02610
  Args:
    request: meeting with design team next Tuesday at 2pm for 1 hour
  manage_email (call_f0ef45111c124f8695f579cc)
 Call ID: call_f0ef45111c124f8695f579cc
  Args:
    request: send them a reminder about reviewing the new mockups
================================= Tool Message =================================
Name: manage_email

Could you please provide the recipient's email address or name so I can send the reminder about reviewing the new mockups?
================================= Tool Message =================================
Name: schedule_event

Perfect! I've successfully scheduled your meeting with the design team for next Tuesday (November 21st) from 2:00 PM to 3:00 PM. The ev

In [14]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

calendar_agent = create_agent(
    model,
    tools = [create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"create_calendar_event": True},
            description_prefix="Calendar event pending approval",
        ),
    ],
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"send_email": True},
            description_prefix="Outbound email pending approval",
        ),
    ],
)

@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })

    return result["messages"][-1].text

@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """

    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })

    return result["messages"][-1].text

supervisor_agent = create_agent(
    model,
    tools = [schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver(),
)

In [15]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": "6"}}

interrupts = []

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================== Ai Message ==================================

I'll help you schedule the meeting with the design team and send them an email reminder about reviewing the mockups.
Tool Calls:
  schedule_event (call_c2373a065d0a401dba42f4e8)
 Call ID: call_c2373a065d0a401dba42f4e8
  Args:
    request: meeting with design team next Tuesday at 2pm for 1 hour
  manage_email (call_aed655ca5aef4a8088d74ec0)
 Call ID: call_aed655ca5aef4a8088d74ec0
  Args:
    request: send them a reminder about reviewing the new mockups
================================= Tool Message =================================
Name: manage_email

Could you please provide the recipient's email address or name so I can send the reminder about reviewing the new mockups?

INTERRUPTED: 95ca3ce574d32a773a1669f4bd5f842c


In [13]:
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")